In [2]:
def cal_result(exp_name, criterion='best', verbose=False):
    import dill as pickle
    data_stream = 70
    week_n = 7000 
    cnt = 0
    AVG = {'tpr':0.0,'fpr':0.0, 'AUC' :0.0} 

    with open('data/y.pkl', 'rb') as f:
        y = pickle.load(f).flatten().tolist()

    for stream in range(1, data_stream + 1): 
        pre = (stream - 1) * 7000
        
        flag = False 
        start, end = pre + week_n, pre + 2 * week_n
        for j in range(start, end): 
            if y[j] == 1: 
                flag = True; break 

        def read_score(exp_name, week): 
            lst = []
            exp_name = os.path.join(os.path.join("exp", exp_name), exp_name)
            with open(exp_name + "Week" + str(week) +  "_result.txt", "r") as f: 
                sc = float(f.readline()) 
                while sc != None: 
                    lst.append(sc)

                    s = f.readline()
                    # while !s: s = f.readline()
                    if s == '': break  
                    sc = float(s) 

            return lst                

        if flag : 
            cnt += 1
            x_score = read_score(exp_name, stream)
            from sklearn.metrics import roc_curve, auc
            GT = y[start: end]
            from sklearn.metrics import roc_auc_score
            AUC = roc_auc_score(GT, x_score)
            if verbose:  print(stream, " week :", AUC)  #*
            
            # print(len(x_score), GT) 
            FPR, TPR, thresholds = roc_curve(GT, x_score)
            maxindex = (TPR-FPR).tolist().index(max(TPR-FPR))
            threshold = thresholds[maxindex]
            tpr, fpr = TPR[maxindex], FPR[maxindex]
            if verbose: print("tpr: %f , fpr: %f, thresh: %f\n" % (tpr, fpr, threshold)) #*

            for i in AVG.keys(): 
                AVG[i] += eval(i)
            # if criterion == 'bset': 
                # JD = [x - y for x, y in zip(TPR, FPR)]
    for i in AVG.keys():
         AVG[i] /= cnt
    print(AVG)
    return AVG 


In [36]:
cal_result("exp_icarl_all_data_None")


{'tpr': 0.8576397173213972, 'fpr': 0.2481695896176379, 'AUC': 0.8348219737428696}


{'tpr': 0.8576397173213972,
 'fpr': 0.2481695896176379,
 'AUC': 0.8348219737428696}

In [37]:
cal_result("exp_icarl_all_data_None1")


{'tpr': 0.7235216561375504, 'fpr': 0.5952716761471374, 'AUC': 0.4823382299341464}


{'tpr': 0.7235216561375504,
 'fpr': 0.5952716761471374,
 'AUC': 0.4823382299341464}

In [3]:
cal_result("exp_icarl_all_data_None_exp")


{'tpr': 0.8997311519836064, 'fpr': 0.19782794044505075, 'AUC': 0.8896230813987891}


{'tpr': 0.8997311519836064,
 'fpr': 0.19782794044505075,
 'AUC': 0.8896230813987891}

In [30]:
import pandas as pd 
flag = False
df = None
p = {1:"",2:'_1',3:"_3"}
for i in range(1, 4): 
    j = p[i]
    tmp = cal_result("exp_icarl_all_data" + j, verbose=False) 
    if flag != False: df = pd.concat([df, pd.DataFrame(data=tmp, index=[0])], axis=0) 
    else : df = pd.DataFrame(data=tmp, index=[0]); flag = True


{'tpr': 0.8490020343274572, 'fpr': 0.253374548476613, 'AUC': 0.8325270388126781}
{'tpr': 0.8544147194626095, 'fpr': 0.2352490044480786, 'AUC': 0.8442588427212386}
{'tpr': 0.8274044331564153, 'fpr': 0.23730192834814542, 'AUC': 0.829287630276777}


In [31]:
display(df)

,tpr,fpr,AUC
0,0.849002,0.253375,0.832527
0,0.854415,0.235249,0.844259
0,0.827404,0.237302,0.829288


In [32]:
df.describe()

,tpr,fpr,AUC
count,3.000000,3.000000,3.000000
mean,0.843607,0.241975,0.835358
std,0.014290,0.009925,0.007877
min,0.827404,0.235249,0.829288
25%,0.838203,0.236275,0.830907
50%,0.849002,0.237302,0.832527
75%,0.851708,0.245338,0.838393
max,0.854415,0.253375,0.844259


In [33]:
df.describe().loc['mean', :].to_dict()

{'tpr': 0.843607062315494,
 'fpr': 0.24197516042427902,
 'AUC': 0.8353578372702312}